In [ ]:
%matplotlib inline

In [ ]:
import os
from matplotlib import pyplot as plt
import cv2
import numpy as np

def calc_bandwidth(lambd, sigma):
    r = np.pi*sigma/lambd
    c = np.sqrt(np.log(2)/2)
    return np.log2((r + c)/(r - c))

def calc_sigma(lambd, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return lambd * c / np.pi  * (p + 1) / (p - 1)

def calc_lambda(sigma, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return sigma * np.pi / c  * (p - 1) / (p + 1)

In [ ]:
fig_dir = '.'
image_path = 'example_aeroplane_s_000021.png'
img = cv2.imread(image_path)

bandwidths = [1.4]
lambdas = [calc_lambda(2.5, bandwidths[0])]  # [5]

n_thetas = 8
n_psis = 4  # 1, 2, 4
n_gammas = 2

thetas = np.linspace(0, np.pi, n_thetas, endpoint=False)
psis = np.linspace(0, 2*np.pi, n_psis, endpoint=False)
gammas = np.linspace(1, 0, n_gammas, endpoint=False)

convolve = False
labels = False
hypercolumn = True
size = (31, 31)

fontsize = 16  # 20
space = 0.15
width = 12

if hypercolumn:
    ncols = int(np.ceil(len(gammas)*len(psis)*len(thetas)))
    nrows = 1
    width *= 8
    space /= 8
else:
    ncols = len(thetas)
    nrows = int(np.ceil(len(gammas)*len(psis)))
    # nrows = len(psis)


print(f"Total Gabor filters: {ncols*nrows}")

img_scale = 6

if convolve:
    nrows *= 2

i = 0
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', squeeze=False, figsize=(width, width*nrows/ncols))
for gm, gamma in enumerate(gammas):
#     i = 0
#     fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
    for bw in bandwidths:
        for lm, lambd in enumerate(lambdas):
            sigma = calc_sigma(lambd, bw)
    #         lambd = calc_lambda(sigma, bw)
#             fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
#         for gm, gamma in enumerate(gammas):
            for ps, psi in enumerate(psis):
                for th, theta in enumerate(thetas):

                    params = {'ksize': size, 'sigma': sigma, 'theta': theta, 'lambd': lambd, 'gamma': gamma, 'psi': psi}
                    gf = cv2.getGaborKernel(**params, ktype=cv2.CV_32F)

                    row, col = (i//ncols), i%ncols
                    if convolve:
                        row *= 2
                    axes[row, col].imshow(gf, cmap='gray', vmin=-1, vmax=1)
                    axes[row, col].set_xticks([])
                    axes[row, col].set_yticks([])
                    # print(np.amin(gf), np.amax(gf))
#                     simplify(th*np.pi/n_thetas)
                    if labels:
                        if i//ncols == 0:
    #                         axes[row, col].set_title(r"$\theta = {:.3}\pi$".format(theta/np.pi))
                            if th == 0:
                                axes[row, col].set_title(r"$\theta = 0$", fontsize=fontsize)
                            else:
                                axes[row, col].set_title(r"$\theta = \frac{{{}}}{{{}}}\pi$".format(th, n_thetas), fontsize=fontsize)
                        if i%ncols == 0:
    #                         axes[row, col].set_ylabel(r"$\psi = {:.3}\pi, \gamma = {}$".format(psi/np.pi, gamma))  #lambd, sigma))
                            if ps == 0:
                                axes[row, col].set_ylabel(r"$\psi = 0, \gamma = {}$".format(gamma), fontsize=fontsize)
                            else:
                                axes[row, col].set_ylabel(r"$\psi = \frac{{{}}}{{{}}}\pi, \gamma = {}$".format(ps, n_psis, gamma), fontsize=fontsize)

                    if convolve:
                        gf = K.expand_dims(gf, -1)
                        gf = K.expand_dims(gf, -1)
                        # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                        # K.conv2d(image.img_to_array(img), gf)
                        fimg = K.conv2d(img, gf, padding='same')
                        fimg = tf.Session().run(fimg[0,:,:,0])
                        axes[row+1, col].imshow(fimg, cmap='gray', vmin=-img_scale, vmax=img_scale)
                        # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                        axes[row+1, col].set_xticks([])
                        axes[row+1, col].set_yticks([])
                        print(np.amin(fimg), np.amax(fimg))
                    i += 1
    plt.tight_layout()
    plt.subplots_adjust(wspace=space, hspace=space)
    if hypercolumn:
        fname = f"gabor_kernels_hc.png"
    else:
        fname = f"gabor_kernels.png"
    plt.savefig(os.path.join(fig_dir, fname), bbox_inches="tight")  # , additional_artists=[lgd])

In [ ]:
import imageio
anim_dir = 'anim'
if not os.path.isdir(anim_dir):
    os.makedirs(anim_dir)

def get_frame(f, bandwidths, sigmas):
    n_thetas = 8
    n_psis = 4  # 1, 2, 4
    n_gammas = 2

    thetas = np.linspace(0, np.pi, n_thetas, endpoint=False)
    psis = np.linspace(0, 2*np.pi, n_psis, endpoint=False)
    gammas = np.linspace(1, 0, n_gammas, endpoint=False)

    convolve = False
    labels = False
    size = (31, 31)

    ncols = len(thetas)
    nrows = int(np.ceil(len(gammas)*len(psis)))
    
    i = 0
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
    for gm, gamma in enumerate(gammas):
        for bw in bandwidths:
            for sg, sigma in enumerate(sigmas):
                lambd = calc_lambda(sigma, bw)
#             for lm, lambd in enumerate(lambdas):
#                 sigma = calc_sigma(lambd, bw)
                for ps, psi in enumerate(psis):
                    for th, theta in enumerate(thetas):

                        params = {'ksize': size, 'sigma': sigma, 'theta': theta, 'lambd': lambd, 'gamma': gamma, 'psi': psi}
                        gf = cv2.getGaborKernel(**params, ktype=cv2.CV_32F)

                        row, col = (i//ncols), i%ncols
                        if convolve:
                            row *= 2
                        axes[row, col].imshow(gf, cmap='gray', vmin=-1, vmax=1)
                        axes[row, col].set_xticks([])
                        axes[row, col].set_yticks([])

                        if labels:
                            if i//ncols == 0:
                                if th == 0:
                                    axes[row, col].set_title(r"$\theta = 0$", fontsize=fontsize)
                                else:
                                    axes[row, col].set_title(r"$\theta = \frac{{{}}}{{{}}}\pi$".format(th, n_thetas), fontsize=fontsize)
                            if i%ncols == 0:
                                if ps == 0:
                                    axes[row, col].set_ylabel(r"$\psi = 0, \gamma = {}$".format(gamma), fontsize=fontsize)
                                else:
                                    axes[row, col].set_ylabel(r"$\psi = \frac{{{}}}{{{}}}\pi, \gamma = {}$".format(ps, n_psis, gamma), fontsize=fontsize)

                        if convolve:
                            gf = K.expand_dims(gf, -1)
                            gf = K.expand_dims(gf, -1)
                            # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                            # K.conv2d(image.img_to_array(img), gf)
                            fimg = K.conv2d(img, gf, padding='same')
                            fimg = tf.Session().run(fimg[0,:,:,0])
                            axes[row+1, col].imshow(fimg, cmap='gray', vmin=-img_scale, vmax=img_scale)
                            # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                            axes[row+1, col].set_xticks([])
                            axes[row+1, col].set_yticks([])
                            print(np.amin(fimg), np.amax(fimg))
                        i += 1
        plt.tight_layout()
        plt.subplots_adjust(wspace=space, hspace=space)
        plt.savefig(os.path.join(anim_dir, f"gabor_kernels_{f}.png"), bbox_inches="tight")  # , additional_artists=[lgd])
#         plt.close()
    return os.path.join(anim_dir, f"gabor_kernels_{f}.png")


def create_gif(filenames, duration=0.5):
    basename, ext = os.path.splitext(filenames[0])  # os.path.basename(filenames[0])
    output_file, _ = basename.rsplit('_', 1)
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(output_file+'.gif', images, duration=duration)

In [ ]:
sigma = 2.5
for bw in bandwidths:
    print(calc_lambda(sigma, bw))

In [ ]:
import copy
fig_dir = '.'
image_path = 'example_aeroplane_s_000021.png'
img = cv2.imread(image_path)

def_bandwidth = 1.4
range_bandwidth = [1, 1.8]
def_lambda = calc_lambda(2.5, def_bandwidth)
range_lambda = [calc_lambda(1, def_bandwidth), calc_lambda(5, def_bandwidth)]
def_theta = 0
range_theta = [-np.pi/2, np.pi/2]
def_psi = 0
range_psi = [0, 2*np.pi]
def_gamma = 0.5
range_gamma = [1, 0]

bandwidths = np.linspace(1, 1.8, num=9, endpoint=True)
# lambdas = [calc_lambda(2.5, bandwidths[0])]

# frames = []
# for bw, bandwidth in enumerate(bandwidths):
#     frames.append(get_frame(bw, [bandwidth], sigmas=[2.5]))

sigmas = np.linspace(1, 5, 9, endpoint=True)
frames = []
for sg, sigma in enumerate(sigmas):
    frames.append(get_frame(sg, [1.4], [sigma]))

bounce_frames = frames[:]
bounce_frames.extend(reversed(frames))
create_gif(bounce_frames, duration=0.25)

# n_thetas = 8
# n_psis = 4  # 1, 2, 4
# n_gammas = 2

# thetas = np.linspace(0, np.pi, n_thetas, endpoint=False)
# psis = np.linspace(0, 2*np.pi, n_psis, endpoint=False)
# gammas = np.linspace(1, 0, n_gammas, endpoint=False)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

from skimage import data
from skimage.util import img_as_float
from skimage.filters import gabor_kernel


def compute_feats(image, kernels):
    feats = np.zeros((len(kernels), 2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        feats[k, 0] = filtered.mean()
        feats[k, 1] = filtered.var()
    return feats


def match(feats, ref_feats):
    min_error = np.inf
    min_i = None
    for i in range(ref_feats.shape[0]):
        error = np.sum((feats - ref_feats[i, :])**2)
        if error < min_error:
            min_error = error
            min_i = i
    return min_i


# prepare filter bank kernels
kernels = []
for theta in range(4):
    theta = theta / 4. * np.pi
    for sigma in (1, 3):
        for frequency in (0.05, 0.25):
            kernel = np.real(gabor_kernel(frequency, theta=theta,
                                          sigma_x=sigma, sigma_y=sigma))
            kernels.append(kernel)


shrink = (slice(0, None, 3), slice(0, None, 3))
brick = img_as_float(data.load('brick.png'))[shrink]
grass = img_as_float(data.load('grass.png'))[shrink]
wall = img_as_float(data.load('rough-wall.png'))[shrink]
image_names = ('brick', 'grass', 'wall')
images = (brick, grass, wall)

# prepare reference features
ref_feats = np.zeros((3, len(kernels), 2), dtype=np.double)
ref_feats[0, :, :] = compute_feats(brick, kernels)
ref_feats[1, :, :] = compute_feats(grass, kernels)
ref_feats[2, :, :] = compute_feats(wall, kernels)

print('Rotated images matched against references using Gabor filter banks:')

print('original: brick, rotated: 30deg, match result: ', end='')
feats = compute_feats(ndi.rotate(brick, angle=190, reshape=False), kernels)
print(image_names[match(feats, ref_feats)])

print('original: brick, rotated: 70deg, match result: ', end='')
feats = compute_feats(ndi.rotate(brick, angle=70, reshape=False), kernels)
print(image_names[match(feats, ref_feats)])

print('original: grass, rotated: 145deg, match result: ', end='')
feats = compute_feats(ndi.rotate(grass, angle=145, reshape=False), kernels)
print(image_names[match(feats, ref_feats)])


def power(image, kernel):
    # Normalize images for better comparison.
    image = (image - image.mean()) / image.std()
    return np.sqrt(ndi.convolve(image, np.real(kernel), mode='wrap')**2 +
                   ndi.convolve(image, np.imag(kernel), mode='wrap')**2)

# Plot a selection of the filter bank kernels and their responses.
results = []
kernel_params = []
for theta in (0, 1):
    theta = theta / 4. * np.pi
    for frequency in (0.1, 0.4):
        kernel = gabor_kernel(frequency, theta=theta)
        params = 'theta=%d,\nfrequency=%.2f' % (theta * 180 / np.pi, frequency)
        kernel_params.append(params)
        # Save kernel and the power image for each image
        results.append((kernel, [power(img, kernel) for img in images]))

fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(5, 6))
plt.gray()

fig.suptitle('Image responses for Gabor filter kernels', fontsize=12)

axes[0][0].axis('off')

# Plot original images
for label, img, ax in zip(image_names, images, axes[0][1:]):
    ax.imshow(img)
    ax.set_title(label, fontsize=9)
    ax.axis('off')

for label, (kernel, powers), ax_row in zip(kernel_params, results, axes[1:]):
    # Plot Gabor kernel
    ax = ax_row[0]
    ax.imshow(np.real(kernel), interpolation='nearest')
    ax.set_ylabel(label, fontsize=7)
    ax.set_xticks([])
    ax.set_yticks([])

    # Plot Gabor responses with the contrast normalized for each filter
    vmin = np.min(powers)
    vmax = np.max(powers)
    for patch, ax in zip(powers, ax_row[1:]):
        ax.imshow(patch, vmin=vmin, vmax=vmax)
        ax.axis('off')

# plt.show()